In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [2]:

def get_ma_so_link_info(link):
    print(link)
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                # print(số)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'Số'.")
    except AttributeError as e:
            print(f"Lỗi: {e}")
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text:
                ngày = normalize_date(text)
                # print(ngày)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
    except AttributeError as e:
        print(f"Lỗi: {e}")
    info_list = [link.strip(),số,ngày]

    # lấy part 1
    part1 = tables[1].find_all('p')
    part1_part_1  = next((i for i, item in enumerate(part1) if "1. Hàng hóa " in item.get_text(strip=True)), -1)
    part1_part_1_text  = part1[part1_part_1].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_2 = next((i for i, item in enumerate(part1) if "Tên thương mại" in item.get_text(strip=True)), -1)
    part1_part_2_text  = part1[part1_part_2].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_3 = next((i for i, item in enumerate(part1) if "Tên gọi theo cấu tạo" in item.get_text(strip=True)), -1)
    part1_part_3_text  = part1[part1_part_3].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_4 = next((i for i, item in enumerate(part1) if "mã hiệu" in item.get_text(strip=True)), -1)
    part1_part_4_text  = part1[part1_part_4].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_5 = next((i for i, item in enumerate(part1) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
    part1_part_5_text  = part1[part1_part_5].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    info_list.append(part1_part_2_text)
    info_list.append(part1_part_3_text)
    info_list.append(part1_part_4_text)
    info_list.append(part1_part_5_text)

    # allpart
    allpart = tables[1].find_all('p')

    # lấy part 3
    pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
    for p_tag in allpart:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part3 = matches[-1]
    # lấy part 2
    try :
        part2_start = next((i for i, item in enumerate(allpart) if "2. Tóm tắt" in item.get_text(strip=True)), -1)-1
        # print(part2_start)
    except:
        part2_start = next((i for i, item in enumerate(allpart) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
        # print(part2_start)
    part2_end = next((i for i, item in enumerate(allpart) if "3. Kết quả" in item.get_text(strip=True)), -1)
    part2_list = []
    for i in allpart[part2_start+1:part2_end]:
        part2_list.append(i.get_text(strip=True))
    part2_list
    part2_text = '\n'.join(part2_list)

    info_list.append(part2_text)
    info_list.append(part3)
    print(info_list)
    return info_list

In [3]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [4]:
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        part_list.append(i.get_text(strip=True).replace('\r', '').replace('\n', '').strip())
        part_text = '\n'.join(part_list)
    return part_text
        

In [10]:
with open('tesst_link.txt', 'r', encoding='utf-8') as file:
    all_links = file.readlines()

In [11]:
all_links

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5146-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-582199.aspx\n',
 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3409-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-ACC-HHP-Flip-Cover-Protective-E-ENG-UNI-618577.aspx\n',
 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2307-TB-TCHQ-2024-ket-qua-xac-dinh-ma-so-Sac-pin-cho-dien-thoai-di-dong-va-may-tinh-bang-613113.aspx\n',
 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-4239-TB-TCHQ-2021-ket-qua-xac-dinh-truoc-ma-so-Chilly-Delicato-526679.aspx\n',
 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-681-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-Vong-deo-do-song-dien-nao-xung-dien-599826.aspx\n',
 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-4514-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-582077.aspx\n',
 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-149-TB-TCHQ-2023-dinh-chinh-Thong-bao-ket-qua-xac-dinh-tru

In [13]:
all_maso_info_list = []
for link in all_links:
    if 'truoc-ma' in link:
        try:
            info_list = get_ma_so_link_info(link)
            all_maso_info_list.append(info_list)
            # write_data_to_excel(maso_sheet,info_list)
        except:
            print(link + "lỗi")
            info_list = [link,"lỗi"]
            all_maso_info_list.append([link,"lỗi"])
            # write_data_to_excel(phanloai_sheet,info_list)

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5146-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-582199.aspx

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5146-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-582199.aspx', '5146/TB-TCHQ', '05-10-2023', 'Tên thương mại: Poron pad', 'Tên gọi theo cấu tạo, công dụng: Miếng nhựa    Poron từ polyurethan xốp mềm, dính 2 mặt, chưa được gia công thêm, kích thước:    30*12*0.8mm, dùng trong sản xuất, nhằm chống thấm nước, mới 100%', 'Ký, mã hiệu, chủng loại: 540103-000584', 'Nhà sản xuất: Tonly Electronics Holding Limited', '', '3919.90.92']
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3409-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-ACC-HHP-Flip-Cover-Protective-E-ENG-UNI-618577.aspx

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3409-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-ACC-HHP-Flip-Cover-Protective-E-ENG-UNI-618577.aspx
lỗi
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-4239-TB-TCHQ-

In [18]:
get_ma_so_link_info("https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3940-TB-TCHQ-2021-ket-qua-hang-hoa-duoc-xac-dinh-truoc-ma-so-484269.aspx")

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3940-TB-TCHQ-2021-ket-qua-hang-hoa-duoc-xac-dinh-truoc-ma-so-484269.aspx


UnboundLocalError: cannot access local variable 'part3' where it is not associated with a value